In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DELL\\Documents\\summurizer\\Text-summurizer-project\\research'

In [3]:
from  dataclasses import dataclass
from  pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from textsummurizer.constants import *
from textsummurizer.utils.common import read_yaml, create_directories



In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

### Create Components

In [6]:
import os 
import urllib.request as request
import zipfile
from textsummurizer.logging import logger
from textsummurizer.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None        
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


             

### Creating the Pipeline

In [8]:


import os

directory_path = "../config/" # Replace with the directory path you want to list

# Check if the directory exists
if os.path.exists(directory_path) and os.path.isdir(directory_path):
    # List the contents of the directory
    contents = os.listdir(directory_path)
    
    # Print the contents
    for item in contents:
        item_path = os.path.join(directory_path, item)  # Get the full path of the item
        
        if os.path.isfile(item_path):  # Check if it's a file
            print("File:", item)
            
            # Read and print the content of the file
            with open(item_path, 'r') as file:
                file_content = file.read()
                print(file_content)
                
            print()  # Add a new line for separation
            
        elif os.path.isdir(item_path):  # Check if it's a subdirectory
            print("Subdirectory:", item)
            
        else:
            print("Unknown item:", item)
else:
    print("Directory does not exist.")


File: config.yaml
artifacts_root: artifacts
data_ingestion:
  root_dir: artifacts/data_ingestion
  source_URL: https://github.com/shaddydevops/Text-summurizer-project/raw/main/summarizer-data.zip
  local_data_file: artifacts/data_ingestion/data.zip
  unzip_dir: artifacts/data_ingestion

  



In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

<class 'box.box.Box'>
<class 'box.box.Box'>
[2023-07-06 02:13:54,226: INFO: common: created directory at: artifacts]
[2023-07-06 02:13:54,228: INFO: common: created directory at: artifacts/data_ingestion]


NameError: name 'header' is not defined